In [ ]:
import apache_beam as beam

In [ ]:
p1 = beam.pipeline() #creation of pipeline object


#creation of pcoolection that is created after execution of transformation
attendace_count(
    p1
     |beam.io.ReadFromText('sample_data/dept_data.txt')
)


## Types of input transforms (input source)
# ReadFromText
## file_pattern (can contain *.txt to read all the files in one go)
## min_bundles_size -> batching while reading ?
## compression_type
## strip_trailing_newlines
## validate : verify presence of file while pipeline gets created 
## skip_header_lines : ignore headers

#ReadFromAvro
#ReadFromParquet
#ReadFromTFRecord (TFRecord : faster for network transmissions)
#ReadFromMessagingQueues (ReadFromPubSub)


## Types of output transforms 
# WriteToText --> num_of_shards
# WriteToAvro
# WriteToParquet --> row_group_buffer_size & record_batch_size
# WriteToTFRecords
# WriteToPubSub

In [ ]:
# sample writing output

p2 = beam.Pipeline()

# lines = (
#     p2
#     | beam.Create(['line1','line2','line3','line3'])
#     | beam.io.WriteToText('sample_data/outputtest')
# )

#KVP

lines3 = (
    p2
    | beam.Create({'k1':{'name':'me','age':12}})
    # | beam.Map(lambda element:element)
    | beam.io.WriteToText('sample_data/outputtest_kvp')
    
)

p2.run()

## Transforms : Map, FlatMap, Filter

In [ ]:

def SplitRow(element):
    return element.split(',')

def Filter(record):
    return record[3] == 'Accounts'
    
    
p2 = beam.Pipeline()



_ = (

    p2
    | beam.io.ReadFromText('sample_data/dept_data.txt')
    | beam.Map(SplitRow) #takes one element and outputs one element [one to one transformation]
    | beam.Filter(Filter)
    | beam.Map(lambda record : (record[1],1)) #emits kvp
    | beam.CombinePerKey(sum) # groupby , combiner + reducer
    | beam.io.WriteToText('sample_output/test')
)

p2.run()

# flatMap -> emit multiple elements for a single element 
# in the above example Map creates an array of 5 elements, but flatmap would create 5 unique elements

# Lambda function 
# result = lambda x,y : x*y --> x,y are arguments and x*y is expression

In [ ]:
!head -n 20 sample_output/test-00000-of-00001

In [ ]:
# MAP - FLAT MAP - FILTER -- PART 2

In [ ]:

def SplitRow(element):
    return element.split(',')

def Filter(record):
    return record[3] == 'Accounts'
    
    
#p2 = beam.Pipeline()

with beam.Pipeline() as p1:

  _ = (

    p1
    | 'Read from file' >> beam.io.ReadFromText('sample_data/dept_data.txt')
    | 'Splitting based on ,' >>beam.Map(SplitRow) #takes one element and outputs one element [one to one transformation]
    | 'Filter based on accoutns' >> beam.Filter(Filter)
    | beam.Map(lambda record : (record[1],1)) #emits kvp
    | beam.CombinePerKey(sum) # groupby , combiner + reducer
    | beam.io.WriteToText('sample_output/test')
  )

#p2.run()

# flatMap -> emit multiple elements for a single element 
# in the above example Map creates an array of 5 elements, but flatmap would create 5 unique elements

# Lambda function 
# result = lambda x,y : x*y --> x,y are arguments and x*y is expression

In [ ]:
# branching pipelines 
# simple exmaple would be two types of filter leading to two different types of ops
# post branching the pipelines run in parallel independently 

In [ ]:
def SplitRow(element):
    return element.split(',')


with beam.Pipeline() as p1:
    input_collection = (
                            p1
                            |"Read from file" >> beam.io.ReadFromText('sample_data/dept_data.txt')
                            |"split_rows" >> beam.Map(SplitRow)
        
                        )
    
    accounts_filter = (
        input_collection
        | 'Get all dept of accounts' >> beam.Filter(lambda record: record[3] == 'Accounts')
        | 'Set record 1'     >> beam.Map(lambda record : (record[1],1))
        | 'Group and sum' >> beam.CombinePerKey(sum)
        | 'Write' >> beam.io.WriteToText('sample_output/accounts')
    )
    
    hr_filter = (
        input_collection
        | 'Get all dept of hr' >> beam.Filter(lambda record: record[3] == 'HR')
        | 'hr Set record 1'     >> beam.Map(lambda record : ("HR, " + record[1],1))
        | 'hr Group and sum' >> beam.CombinePerKey(sum)
        | 'hr Write' >> beam.io.WriteToText('sample_output/hr')

    )

In [ ]:
# Flatten : Union operation (combines P collections
# sample data type and cols required

In [ ]:
def SplitRow(element):
    return element.split(',')


with beam.Pipeline() as p1:
    input_collection = (
                            p1
                            |"Read from file" >> beam.io.ReadFromText('sample_data/dept_data.txt')
                            |"split_rows" >> beam.Map(SplitRow)
        
                        )
    
    accounts_filter = (
        input_collection
        | 'Get all dept of accounts' >> beam.Filter(lambda record: record[3] == 'Accounts')
        | 'Set record 1'     >> beam.Map(lambda record : ('accounts' + record[1],1))
        | 'Group and sum' >> beam.CombinePerKey(sum)
        #| 'Write' >> beam.io.WriteToText('sample_output/accounts')
    )
    
    hr_filter = (
        input_collection
        | 'Get all dept of hr' >> beam.Filter(lambda record: record[3] == 'HR')
        | 'hr Set record 1'     >> beam.Map(lambda record : ('HR' + record[1],1))
        | 'hr Group and sum' >> beam.CombinePerKey(sum)
        #| 'hr Write' >> beam.io.WriteToText('sample_output/hr')

    )
    
    output = (
        (accounts_filter,hr_filter)
        |beam.Flatten()
        |beam.io.WriteToText('sample_output/flatten')
        
    
    )

In [ ]:
# PARDO Transformation
# parallel processing
# takes element -> proccesses -> return 
# parent calss for map and flatmap
# find out more on pardo 
# contains dofn ... for distributed compute implementation .. create a class 

In [ ]:
class SplitRow(beam.DoFn): #inherits DoFn
    def process(self,element):
        return [element.split(',')]
    

    
with beam.Pipeline() as p1:
    input_collection = (
                            p1
                            |"Read from file" >> beam.io.ReadFromText('sample_data/dept_data.txt')
                            |"split_rows" >> beam.ParDo(SplitRow())
                            |beam.io.WriteToText('sample_output/pardotest')
        
                        )
#  filter and other transformation can be specified in pardo 


In [ ]:
# using pardo and grouping

def SplitRow(element):
    return element.split(',')


class GroupSum(beam.DoFn):
    def process(self,element):
        (key,value) = element
        return [(key,sum(value))]

# class grouping(beam.DoFn):
#     def process(self,element):
        
with beam.Pipeline() as p1:
    input_collection = (
                            p1
                            |"Read from file" >> beam.io.ReadFromText('sample_data/dept_data.txt')
                            |"split_rows" >> beam.Map(SplitRow)
        
                        )
    
    accounts_filter = (
        input_collection
        | 'Get all dept of accounts' >> beam.Filter(lambda record: record[3] == 'Accounts')
        | 'Set record 1'     >> beam.Map(lambda record : (record[1],1))
        | 'group' >> beam.GroupByKey()
        | 'Group sum' >> beam.ParDo(GroupSum())
        | 'Write' >> beam.io.WriteToText('sample_output/ltest1')
    )
